
## Steam Silver Layer

Esse notebook vai anexar as funcionalidades do Steam Bronze Layer, visto que a versão Community do Databricks não tem a opção de criar um Workflow. Além disso, esse notebook também vai limpar, filtrar e persistir os dados no [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) para que seja possível fazer análises na próxima camada.

In [0]:
%run "./01_steam_bronze_layer_notebook"

count(1)
83646


In [0]:
from pyspark.sql import SparkSession

# Inicializando a Sessão do Spark
spark = SparkSession.builder \
    .appName("Read Parquet from DBFS") \
    .getOrCreate()

# Localização do arquivo
file_location = "dbfs:/user/hive/warehouse/steam_bronze_layer_table"

# Lendo o Parquet
df = spark.read.parquet(file_location)

In [0]:
# Mostrando o DataFrame lido do parquet da Bronze Layer
df.show()

+-------+--------------------+------------+------------+-----+---------+-------------------------------------+-------------------------------------+----------------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+-----+-----+----------------+--------------------+------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+----------+--------+--------+----------------+------------------------+-----------------------+-----------------------+----------------------+--------+--------------------+-------------+-----------------+--------------+------------------+
|  AppID|                name|release_date|required_age|price|dlc_count|                 detailed_description|                       about_the_game|                 

In [0]:
# Print the schema
df.printSchema()

root
 |-- AppID: string (nullable = true)
 |-- name: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- required_age: string (nullable = true)
 |-- price: string (nullable = true)
 |-- dlc_count: string (nullable = true)
 |-- detailed_description: string (nullable = true)
 |-- about_the_game: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- reviews: string (nullable = true)
 |-- header_image: string (nullable = true)
 |-- website: string (nullable = true)
 |-- support_url: string (nullable = true)
 |-- support_email: string (nullable = true)
 |-- windows: string (nullable = true)
 |-- mac: string (nullable = true)
 |-- linux: string (nullable = true)
 |-- metacritic_score: string (nullable = true)
 |-- metacritic_url: string (nullable = true)
 |-- achievements: string (nullable = true)
 |-- recommendations: string (nullable = true)
 |-- notes: string (nullable = true)
 |-- supported_languages: string (nullable = true)
 |-- full_audio

In [0]:
# Selecionando apenas as colunas que serão utilizadas

df = df.select("AppID", "name", "release_date", "price", "dlc_count", "achievements", "peak_ccu")

df.show()
df.printSchema()

+-------+--------------------+------------+-----+---------+------------+--------+
|  AppID|                name|release_date|price|dlc_count|achievements|peak_ccu|
+-------+--------------------+------------+-----+---------+------------+--------+
|  62100|               Chime|  2010-09-06| 4.99|        0|          15|       3|
|2479830|          The Braves|  2024-05-03|  0.0|        0|         112|       0|
| 320760|   Tokyo School Life|  2015-02-12|14.99|        1|           0|       1|
|1710920|    Dwerve: Prologue|  2021-09-03|  0.0|        0|           0|       0|
|2055220|  Desktop Girlfriend|  2022-08-23| 1.99|        1|          71|       1|
| 384680|           MechaNika|  2015-07-16| 3.99|        1|          30|       1|
|2824230|    last seen online|  2024-03-14|  0.0|        0|          11|       0|
| 946090|    Hentai Neighbors|  2018-10-17| 0.49|        1|         100|       0|
|1430680|Clash: Artifacts ...|  2023-03-09|29.99|        4|          21|       9|
|1266540|       

In [0]:
# Renomeando as colunas

df = df.withColumnRenamed("AppID", "STEAM_ID") \
       .withColumnRenamed("name", "TITLE_NAME") \
       .withColumnRenamed("release_date", "RELEASE_DATE") \
       .withColumnRenamed("price", "PRICE") \
       .withColumnRenamed("dlc_count", "DLC_COUNT") \
       .withColumnRenamed("achievements", "ACHIEVEMENT") \
       .withColumnRenamed("peak_ccu", "PEAK_CONCURRENT_USERS")

df.show()
df.printSchema()

+--------+--------------------+------------+-----+---------+-----------+---------------------+
|STEAM_ID|          TITLE_NAME|RELEASE_DATE|PRICE|DLC_COUNT|ACHIEVEMENT|PEAK_CONCURRENT_USERS|
+--------+--------------------+------------+-----+---------+-----------+---------------------+
|   62100|               Chime|  2010-09-06| 4.99|        0|         15|                    3|
| 2479830|          The Braves|  2024-05-03|  0.0|        0|        112|                    0|
|  320760|   Tokyo School Life|  2015-02-12|14.99|        1|          0|                    1|
| 1710920|    Dwerve: Prologue|  2021-09-03|  0.0|        0|          0|                    0|
| 2055220|  Desktop Girlfriend|  2022-08-23| 1.99|        1|         71|                    1|
|  384680|           MechaNika|  2015-07-16| 3.99|        1|         30|                    1|
| 2824230|    last seen online|  2024-03-14|  0.0|        0|         11|                    0|
|  946090|    Hentai Neighbors|  2018-10-17| 0.49|

In [0]:
from pyspark.sql.types import DateType, DoubleType, IntegerType

# Alterando o tipo de dados das colunas que estão com o tipo errado

df = df.withColumn("RELEASE_DATE", df["RELEASE_DATE"].cast(DateType()))
df = df.withColumn("PRICE", df["PRICE"].cast(DoubleType()))
df = df.withColumn("DLC_COUNT", df["DLC_COUNT"].cast(IntegerType()))
df = df.withColumn("ACHIEVEMENT", df["ACHIEVEMENT"].cast(IntegerType()))
df = df.withColumn("PEAK_CONCURRENT_USERS", df["PEAK_CONCURRENT_USERS"].cast(IntegerType()))

df.show()
df.printSchema()

+--------+--------------------+------------+-----+---------+-----------+---------------------+
|STEAM_ID|          TITLE_NAME|RELEASE_DATE|PRICE|DLC_COUNT|ACHIEVEMENT|PEAK_CONCURRENT_USERS|
+--------+--------------------+------------+-----+---------+-----------+---------------------+
|   62100|               Chime|  2010-09-06| 4.99|        0|         15|                    3|
| 2479830|          The Braves|  2024-05-03|  0.0|        0|        112|                    0|
|  320760|   Tokyo School Life|  2015-02-12|14.99|        1|          0|                    1|
| 1710920|    Dwerve: Prologue|  2021-09-03|  0.0|        0|          0|                    0|
| 2055220|  Desktop Girlfriend|  2022-08-23| 1.99|        1|         71|                    1|
|  384680|           MechaNika|  2015-07-16| 3.99|        1|         30|                    1|
| 2824230|    last seen online|  2024-03-14|  0.0|        0|         11|                    0|
|  946090|    Hentai Neighbors|  2018-10-17| 0.49|

In [0]:
# Validação no STEAM_ID

# Garantindo que não tem ID repetido

distinct_values_df = df.groupBy("STEAM_ID").count()
distinct_values_df.filter(distinct_values_df["count"] > 1).show()

# Garantindo que não tem ID negativo
df.filter(df["STEAM_ID"] < 0).show()
df.filter(df["STEAM_ID"].isNull()).show()

+--------+-----+
|STEAM_ID|count|
+--------+-----+
+--------+-----+

+--------+----------+------------+-----+---------+-----------+---------------------+
|STEAM_ID|TITLE_NAME|RELEASE_DATE|PRICE|DLC_COUNT|ACHIEVEMENT|PEAK_CONCURRENT_USERS|
+--------+----------+------------+-----+---------+-----------+---------------------+
+--------+----------+------------+-----+---------+-----------+---------------------+

+--------+----------+------------+-----+---------+-----------+---------------------+
|STEAM_ID|TITLE_NAME|RELEASE_DATE|PRICE|DLC_COUNT|ACHIEVEMENT|PEAK_CONCURRENT_USERS|
+--------+----------+------------+-----+---------+-----------+---------------------+
+--------+----------+------------+-----+---------+-----------+---------------------+



In [0]:
# Validação no TITLE_NAME

# Garantindo que não tem nome vázio
df_filtered = df.filter(df["TITLE_NAME"] == "")
print(df_filtered.count())

# Removendo os Títulos NULL, pois não tem como saber quais são
df.filter(df["TITLE_NAME"].isNull()).show()
df = df.dropna(subset=['TITLE_NAME'])
df.filter(df["TITLE_NAME"].isNull()).show()


0
+--------+----------+------------+-----+---------+-----------+---------------------+
|STEAM_ID|TITLE_NAME|RELEASE_DATE|PRICE|DLC_COUNT|ACHIEVEMENT|PEAK_CONCURRENT_USERS|
+--------+----------+------------+-----+---------+-----------+---------------------+
|  396420|      null|  2016-11-01|  0.0|        0|          0|                    0|
| 1116910|      null|  2019-09-25| 2.79|        0|          0|                    0|
| 1347240|      null|  2021-04-20|24.99|        0|          0|                    0|
+--------+----------+------------+-----+---------+-----------+---------------------+

+--------+----------+------------+-----+---------+-----------+---------------------+
|STEAM_ID|TITLE_NAME|RELEASE_DATE|PRICE|DLC_COUNT|ACHIEVEMENT|PEAK_CONCURRENT_USERS|
+--------+----------+------------+-----+---------+-----------+---------------------+
+--------+----------+------------+-----+---------+-----------+---------------------+



In [0]:
from datetime import date

# Validação no RELEASE_DATE

reference_date = date(2024, 7, 7)

# Garantindo que não tem data no futuro
df_filtered = df.filter(df["RELEASE_DATE"] > reference_date)
print(df_filtered.count())

# Removendo os REFERENCE_DATE NULL, pois não tem como saber quando foi
df.filter(df["RELEASE_DATE"].isNull()).show()
df = df.dropna(subset=['RELEASE_DATE'])
df.filter(df["RELEASE_DATE"].isNull()).show()


0
+--------+---------------------+------------+-----+---------+-----------+---------------------+
|STEAM_ID|           TITLE_NAME|RELEASE_DATE|PRICE|DLC_COUNT|ACHIEVEMENT|PEAK_CONCURRENT_USERS|
+--------+---------------------+------------+-----+---------+-----------+---------------------+
| 2348100| "YEAH! YOU WANT "...|        null| null|        0|          0|                 null|
| 2264930|"『LACKGIRL I - ""...|        null|  0.0|       16|       null|                   41|
| 1176040| "I Have Low Stats...|        null|  0.0|       14|       null|                    7|
|  817820|         "The ""Quiet|        null|  0.0|        4|       null|                   10|
+--------+---------------------+------------+-----+---------+-----------+---------------------+

+--------+----------+------------+-----+---------+-----------+---------------------+
|STEAM_ID|TITLE_NAME|RELEASE_DATE|PRICE|DLC_COUNT|ACHIEVEMENT|PEAK_CONCURRENT_USERS|
+--------+----------+------------+-----+---------+----------

In [0]:
# VALIDAÇÃO NO PRICE

# Garantindo que não tem valores negativos
df.filter(df["PRICE"].isNull()).show()
df.filter(df["PRICE"] < 0).show()

+--------+----------+------------+-----+---------+-----------+---------------------+
|STEAM_ID|TITLE_NAME|RELEASE_DATE|PRICE|DLC_COUNT|ACHIEVEMENT|PEAK_CONCURRENT_USERS|
+--------+----------+------------+-----+---------+-----------+---------------------+
+--------+----------+------------+-----+---------+-----------+---------------------+

+--------+----------+------------+-----+---------+-----------+---------------------+
|STEAM_ID|TITLE_NAME|RELEASE_DATE|PRICE|DLC_COUNT|ACHIEVEMENT|PEAK_CONCURRENT_USERS|
+--------+----------+------------+-----+---------+-----------+---------------------+
+--------+----------+------------+-----+---------+-----------+---------------------+



In [0]:
# VALIDAÇÃO NO DLC_COUNT

distinct_values_df = df.groupBy("DLC_COUNT").count()
distinct_values_df.show()

# Garantindo que não tem números negativos de DLC
df.filter(df["DLC_COUNT"].isNull()).show()
df.filter(df["DLC_COUNT"] < 0).show()

+---------+-----+
|DLC_COUNT|count|
+---------+-----+
|       34|    2|
|       26|   12|
|       27|    5|
|       12|   50|
|       22|   15|
|        1| 8527|
|       13|   40|
|        6|  189|
|       16|   21|
|        3|  846|
|       20|   12|
|        5|  300|
|       19|   18|
|       15|   27|
|       43|    5|
|       37|    6|
|        9|   73|
|       17|   21|
|       35|    7|
|      278|    1|
+---------+-----+
only showing top 20 rows

+--------+----------+------------+-----+---------+-----------+---------------------+
|STEAM_ID|TITLE_NAME|RELEASE_DATE|PRICE|DLC_COUNT|ACHIEVEMENT|PEAK_CONCURRENT_USERS|
+--------+----------+------------+-----+---------+-----------+---------------------+
+--------+----------+------------+-----+---------+-----------+---------------------+

+--------+----------+------------+-----+---------+-----------+---------------------+
|STEAM_ID|TITLE_NAME|RELEASE_DATE|PRICE|DLC_COUNT|ACHIEVEMENT|PEAK_CONCURRENT_USERS|
+--------+----------+----------

In [0]:
# VALIDAÇÃO NO ACHIEVEMENT

# Garantindo que não tem valores negativos
df.filter(df["ACHIEVEMENT"] < 0).show()

# Removendo os ACHIEVEMENT NULL, pois não tem como saber quantos foram
df.filter(df["ACHIEVEMENT"].isNull()).show()
df = df.dropna(subset=['ACHIEVEMENT'])
df.filter(df["ACHIEVEMENT"].isNull()).show()

+--------+----------+------------+-----+---------+-----------+---------------------+
|STEAM_ID|TITLE_NAME|RELEASE_DATE|PRICE|DLC_COUNT|ACHIEVEMENT|PEAK_CONCURRENT_USERS|
+--------+----------+------------+-----+---------+-----------+---------------------+
+--------+----------+------------+-----+---------+-----------+---------------------+

+--------+--------------------+------------+-----+---------+-----------+---------------------+
|STEAM_ID|          TITLE_NAME|RELEASE_DATE|PRICE|DLC_COUNT|ACHIEVEMENT|PEAK_CONCURRENT_USERS|
+--------+--------------------+------------+-----+---------+-----------+---------------------+
|  319970|          The Detail|  2014-10-28| 2.99|        2|       null|                  182|
|   99100|Dungeons and Drag...|  2011-06-24|14.99|        0|       null|                 null|
|  223450|                Dyad|  2013-04-24|14.99|        0|       null|                    0|
|  209790|              Splice|  2012-06-13| 9.99|        2|       null|                 

In [0]:
# VALIDAÇÃO NO PEAK_CONCURRENT_USERS

# Garantindo que não tem valores negativos
df.filter(df["PEAK_CONCURRENT_USERS"] < 0).show()

# Removendo os PEAK_CONCURRENT_USERS NULL, pois não tem como saber quantos foram
df.filter(df["PEAK_CONCURRENT_USERS"].isNull()).show()
df = df.dropna(subset=['PEAK_CONCURRENT_USERS'])
df.filter(df["PEAK_CONCURRENT_USERS"].isNull()).show()

+--------+----------+------------+-----+---------+-----------+---------------------+
|STEAM_ID|TITLE_NAME|RELEASE_DATE|PRICE|DLC_COUNT|ACHIEVEMENT|PEAK_CONCURRENT_USERS|
+--------+----------+------------+-----+---------+-----------+---------------------+
+--------+----------+------------+-----+---------+-----------+---------------------+

+--------+--------------------+------------+-----+---------+-----------+---------------------+
|STEAM_ID|          TITLE_NAME|RELEASE_DATE|PRICE|DLC_COUNT|ACHIEVEMENT|PEAK_CONCURRENT_USERS|
+--------+--------------------+------------+-----+---------+-----------+---------------------+
| 2204230|  MineSweeper Tetris|  2022-12-15|  0.0|        0|         26|                 null|
| 1334000|              Wordle|  2021-04-12|19.99|        4|          6|                 null|
| 1091980|The Bard's Tale I...|  2019-08-27|34.99|        0|         59|                 null|
| 2803010|       100 Robo Cats|  2024-02-26|  0.0|        3|        100|                 

In [0]:
%py

# Configuração para habilitar o Overwrite quando o Cluster for desligado
spark.conf.set("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation", "true")

In [0]:
# Criação da tabela persistida usando o CREATE OR ALTER do PySpark

permanent_table_name = "steam_silver_layer_table"
df.write.mode("overwrite").format("parquet").saveAsTable(permanent_table_name)

In [0]:
%sql

/* Checagem para ver se os dados inseridos podem ser acessados usando o SparkSQL. */

select * from `steam_silver_layer_table` limit 5;

STEAM_ID,TITLE_NAME,RELEASE_DATE,PRICE,DLC_COUNT,ACHIEVEMENT,PEAK_CONCURRENT_USERS
880330,Last Byte Standing,2018-07-30,14.99,1,89,0
1756340,FishWitch Halloween,2021-10-25,9.99,0,0,0
1416930,Infected Prison,2021-02-04,1.99,0,0,0
2230790,Scary pictures: Yavez - seven deadly sins,2022-12-14,1.19,0,18,0
1169540,HIGHRISE,2019-10-30,2.99,0,16,0


In [0]:
%sql

/* Sobraram 82472 registros após a limpeza. Resultando numa remoção de 1174 linhas. */

select count(*) from `steam_silver_layer_table`

count(1)
82472
